### **"비만", "비만 관련 질병" 이라는 키워드로 네이버 검색 엔진에서 크롤링한 결과**

In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

url = "https://search.naver.com/search.naver"

data_1 = []

for i in range(0,9):
    start_num = i * 15 + 1  # 각 페이지의 시작점 계산 - 네이버에서는 페이지를 넘어갈 때 마다 나타내는 수가 15씩 증가하는 걸로 확인하였다.
    params = {
        'where': 'web',
        'query': '비만',  # 검색어 설정
        'start': start_num  # 페이지 시작점
    }
    req = requests.get(url, params)
    if req.encoding != 'UTF-8' and req.encoding != 'EUC-KR':
                req.encoding = 'UTF-8'

    soup = BeautifulSoup(req.text, 'html.parser')
    
    news_link = soup.find_all('a', class_='link_tit')    
    
    for link in news_link:
        title = link.text
        n_url = link.get('href')
        print(title, n_url)

        try:
            new_req = requests.get(n_url)
            n_soup = BeautifulSoup(new_req.text, 'html.parser')
        
            content = n_soup.get_text()
            keywords = re.findall(r"비만[^\.]*?(?:원인|요인|결과|영향|유발|중요|있습니다|질병|이다|니다)[^\.]*\.",content)

            for keyword in keywords:
                data_1.append([title, n_url, keyword])
                
        except:
            continue

비만 | 질환백과 | 의료정보 | 건강정보 | 서울아산병원 https://www.amc.seoul.kr/asan/healthinfo/disease/diseaseDetail.do?contentId=31809
비만 - 나무위키 https://namu.wiki/w/%EB%B9%84%EB%A7%8C
비만 - 영양 장애 - MSD 매뉴얼 - 일반인용 https://www.msdmanuals.com/ko/home/%EC%98%81%EC%96%91-%EC%9E%A5%EC%95%A0/%EB%B9%84%EB%A7%8C%EA%B3%BC-%EB%8C%80%EC%82%AC-%EC%A6%9D%ED%9B%84%EA%B5%B0/%EB%B9%84%EB%A7%8C
비만의 6가지 유형과 원인 - 건강을 위한 발걸음 https://steptohealth.co.kr/6-obesity-and-their-causes/
비만이란? https://www.novonordisk.co.kr/patient/obesity-treatment-products/define-baseline.html
주간 감염병 최신 정보 5권 37호 ('20.9.14) - 소식지 - 삼성서울병원 http://www.samsunghospital.com/home/info/letterView.do?SEQ=196
소아 비만 | 국가건강정보포털 | 질병관리청 https://health.kdca.go.kr/healthinfo/biz/health/gnrlzHealthInfo/gnrlzHealthInfo/gnrlzHealthInfoView.do?cntnts_sn=4008
비만 세포 - 위키백과 한국어 https://ko.wikipedia.org/wiki/%EB%B9%84%EB%A7%8C_%EC%84%B8%ED%8F%AC
비만증 - 위키백과 한국어 https://ko.wikipedia.org/wiki/%EB%B9%84%EB%A7%8C%EC%A6%9D
비만 | 국가건강정보포털 | 질병관리청 https://health.kdca.go.kr/healthin

----
"비만"이라는 키워드로 네이버 검색 엔진에서 검색했을 때 나온 결과들 중에서 link_tit이라는 class를 가진 링크들을 크롤링하여 해당 링크의 title, URL, text 등을 획득하였다.

비만이라는 단어가 포함된 문장이 필요한 정보를 담고 있는 문장일 것이라고 판단하였다.<br>
text에서 필요한 정보만을 얻기 위해, 정규표현식을 이용하여, 비만이라는 단어를 포함하는 문장들만 추출하였다.

----

In [3]:
from konlpy.tag import Okt
from collections import Counter

# 질병 이름 목록 (예시, 실제 목록은 더 포괄적일 수 있음)
disease_names = [
    "비만", "당뇨병", "고혈압", "심장병", "뇌졸중", "암", "관절염", "간경변", "고지혈증", "우울증", 
    "불면증", "천식", "폐렴", "치매", "골다공증", "위염", "신부전", "간암", "위암", "대장암"
]

df = pd.DataFrame(data_1, columns=['Title', 'URL', 'Sentence'])

# df.to_excel('obesity_related_diseases_data.xlsx', index=False) 
# 위 코드로 크롤링 결과를 excel로 저장할 수 있다.

okt = Okt()
all_sentences = ' '.join(df['Sentence'])
nouns = okt.nouns(all_sentences)

# 불용어 제거
stop_words = ['관련', '질병', '유발', '야기']
nouns = [word for word in nouns if (word not in stop_words) and (word in disease_names)]

# 빈도수 계산
counter = Counter(nouns)
most_common_words = counter.most_common()

# 결과 출력
print("가장 많이 언급된 질병:")
for word, freq in most_common_words:
    if freq >= 5:
        print(f"{word}: {freq}회")
    else:
        pass

가장 많이 언급된 질병:
비만: 2696회
당뇨병: 113회
고혈압: 92회
치매: 42회
뇌졸중: 26회
관절염: 24회
우울증: 24회
고지혈증: 21회
암: 20회
천식: 14회
골다공증: 9회
폐렴: 7회
심장병: 6회


----
잘 알려진 질병의 이름을 담은 리스트를 직접 생성하였다.<br>
"비만"이라는 키워드로 검색했을 때 나온 링크를 크롤링하여 추출한 문장들에서, 해당 리스트에 있는 질병들이 얼마나 빈번하게 언급되었는지 출력하는 코드이다.

검색 키워드인 비만을 제외하면, 당뇨병, 고혈압, 치매, 뇌졸중 등이 가장 빈번하게 나타났음을 알 수 있다.

----

In [4]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

url = "https://search.naver.com/search.naver"

data_2 = []

for i in range(0,9):
    start_num = i * 15 + 1  # 각 페이지의 시작점 계산 - 네이버에서는 페이지를 넘어갈 때 마다 나타내는 수가 15씩 증가하는 걸로 확인하였다.
    params = {
        'where': 'web',
        'query': '비만+관련+질병',  # 검색어 설정
        'start': start_num  # 페이지 시작점
    }
    req = requests.get(url, params)
    if req.encoding != 'UTF-8' and req.encoding != 'EUC-KR':
                req.encoding = 'UTF-8'

    soup = BeautifulSoup(req.text, 'html.parser')

    news_link = soup.find_all('a', class_='link_tit')
    
    for link in news_link:
        title = link.text
        n_url = link.get('href')
        print(title, n_url)

        try:
            new_req = requests.get(n_url)
            n_soup = BeautifulSoup(new_req.text, 'html.parser')
        
            content = n_soup.get_text()
            keywords = re.findall(r"비만[^\.]*?(?:유발|야기)[^\.]*\.",content)


            for keyword in keywords:
                data_2.append([title, n_url, keyword])
                
        except:
            continue

비만 - 나무위키 https://namu.wiki/w/%EB%B9%84%EB%A7%8C
비만 | 질환백과 | 의료정보 | 건강정보 | 서울아산병원 https://www.amc.seoul.kr/asan/healthinfo/disease/diseaseDetail.do?contentId=31809
비만의 동반질환 - 대한비만학회 https://general.kosso.or.kr/html/?pmode=obesityDisease
소아 비만과 관련한 질병 8가지 - 건강을 위한 발걸음 https://steptohealth.co.kr/8-diseases-related-to-childhood-obesity/
강아지 비만도 표준 기준 관련 질병 https://blog.naver.com/kkh5981/223308955494
비만 | 국가건강정보포털 | 질병관리청 https://health.kdca.go.kr/healthinfo/biz/health/gnrlzHealthInfo/gnrlzHealthInfo/gnrlzHealthInfoView.do?cntnts_sn=5292
"흡연·비만, 류마티스관절염 질병 활성도와 관련 없어" < 소화기/류마티스 < 학술 < 기사본문 - 메디칼업저버 http://www.monews.co.kr/news/articleView.html?idxno=214769
비만은 질병이다, ‘만성질환의 씨앗’ 비만 다스리기 - 당신의 건강가이드 헬스조선 https://health.chosun.com/site/data/html_dir/2016/06/13/2016061301044.html
다이어트 필수 정보 !! 비만의 원인과 관련 질병 https://ben-jh-family-history.tistory.com/entry/%EB%B9%84%EB%A7%8C%EC%9D%B4-%EC%9B%90%EC%9D%B8%EC%9D%B4-%EB%90%98%EB%8A%94-%EC%97%AC%EB%9F%AC%EA%B0%80%EC%A7%80-%EC%A7%88%EB%B3%91%EB%93%A4%E

----
"비만 관련 질병" 이라는 키워드로 네이버 검색 엔진에서 검색했을 때 나온 결과들 중에서, link_tit이라는 class를 가진 링크들을 크롤링하여 해당 링크의 title, URL, text 등을 획득하였다.

마찬가지로, text에서 필요한 정보만을 얻기 위해 정규표현식을 이용하여, 비만이라는 단어를 포함하는 문장들만 추출하였다.

----

In [5]:
from konlpy.tag import Okt
from collections import Counter

# 질병 이름 목록 (예시, 실제 목록은 더 포괄적일 수 있음)
disease_names = [
    "비만", "당뇨병", "고혈압", "심장병", "뇌졸중", "암", "관절염", "간경변", "고지혈증", "우울증", 
    "불면증", "천식", "폐렴", "치매", "골다공증", "위염", "신부전", "간암", "위암", "대장암"
]

df = pd.DataFrame(data_2, columns=['Title', 'URL', 'Sentence'])

# df.to_excel('obesity_related_diseases_data_2.xlsx', index=False) 
# 위 코드로 크롤링 결과를 excel로 저장할 수 있다.

okt = Okt()
all_sentences = ' '.join(df['Sentence'])
nouns = okt.nouns(all_sentences)

# 불용어 제거
stop_words = ['관련', '질병', '유발', '야기']
nouns = [word for word in nouns if (word not in stop_words) and (word in disease_names)]

# 빈도수 계산
counter = Counter(nouns)
most_common_words = counter.most_common()

# 결과 출력
print("가장 많이 나온 질병:")
for word, freq in most_common_words:
    if freq >= 5: # 5번 이상 나온 단어만 출력하였다.
        print(f"{word}: {freq}회")
    else:
        pass

가장 많이 나온 질병:
비만: 200회
당뇨병: 19회
고혈압: 17회
암: 15회
고지혈증: 15회


----
"비만 관련 질병"이라는 키워드로 검색했을 때 나온 링크를 크롤링하여 추출한 문장들에서, 해당 리스트에 있는 질병들이 얼마나 빈번하게 언급되었는지 출력하는 코드이다.

해당 키워드로 크롤링한 결과, 비만을 제외하면 당뇨병, 암, 고혈압, 고지혈증 등이 가장 빈번하게 언급된 것을 확인할 수 있다. 이 중, 암은 비만 외에도 다양한 원인으로 발병하기 때문에 비만과의 관련성을 단독으로 분석하기에는 어려움이 있다. 따라서, 두 크롤링 결과에서 공통적으로 빈번하게 언급된 질병인 당뇨병, 고혈압, 고지혈증에 대한 분석을 진행하고자 한다.

----